In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.types import *
from delta import *

In [ ]:
builder = SparkSession.builder.appName("olist_demo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.executor.memory", "2g")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.memory", "4g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Config

In [3]:
environment = '' # dev;uat;prd

In [ ]:
base_path = '../../../data'  # This will be overwritten by Papermill

In [4]:
path = {
    'slv': {'order_reviews': f'{base_path}/silver/evt/evt_order_reviews'}, 
    'snk': f'{base_path}/gold/fct/fct_order_reviews'
}

## Load

In [5]:
df = spark.read.format('delta').load(path['slv']['order_reviews'])

In [6]:
df_snk = df.select(
    F.col('id_review'),
    F.col('id_order'),
    F.col('nr_review_score'),
    F.col('ds_review_comment_title'),
    F.col('ds_review_comment_message'),
    F.col('dt_review_creation'),
    F.col('dh_review_answer'),
)

In [7]:
df_snk = df_snk.withColumn("dh_insert_gld", F.current_timestamp())

## Sink

In [ ]:
df_snk.write.format('delta')\
.mode(saveMode='overwrite')\
.save(path['snk'])